# IMPORT LIBARAY

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import datetime
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from scipy import interpolate

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import plotly.graph_objects as go
from matplotlib import dates
from plotly.subplots import make_subplots
import plotly
import plotly.express as px
pd.options.display.float_format = '{:.5f}'.format

import FinanceDataReader as fdr

from sklearn.preprocessing import MinMaxScaler

# os.getcwd()
### 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
font_list
plt.rc("font", family="DejaVu Sans")
parameters = {'axes.labelsize': 20,
          'axes.titlesize': 25,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
        'legend.fontsize': 20,
          }
plt.rcParams.update(parameters)
plt.rcParams['font.family'] = 'DejaVu Sans'

current_dir = os.getcwd()
plt.style.use("seaborn-dark")
plt.rcParams['font.family'] = 'DejaVu Sans'

In [2]:
# dir_root = '../../../'
# current_dir = os.getcwd()

# dir_master = os.path.join(dir_root, '01. Data','00. master_data')
# dir_processed = os.path.join(dir_root, '01. Data','02. 재무지표2', 'ITS')
# dir_result = os.path.join(current_dir, 'result')


# root_path = "C:/Users/10188/Desktop/기업가치"
# master_path = os.path.join(root_path, "master_data")
# rawfile_path = os.path.join(master_path, "result_xlsx")

# out_path = os.path.join(root_path, "clean_data","clean_total")




root_path = '../../../'
current_dir = os.getcwd()

master_path = os.path.join(root_path, '01. Data','00. master_data')
rawfile_path = os.path.join(master_path, "result_xlsx")

file_path = os.path.join(root_path, '01. Data','02. 재무지표2', '01. preprocessed')
final_data_path = os.path.join(root_path, '01. Data','02. 재무지표2', '02. feature_selected')

result_path = os.path.join(current_dir, 'result')

In [3]:
desc = pd.read_excel(os.path.join(master_path, "desc_v2.2.xlsx"), sheet_name='desc', header=0, usecols=['no','raw','variable','group','importance'])
desc.head(2)

,raw,no,variable,group,importance
0,Date_x,1,Date_x,DT,9
1,BS_Earnings Quality Score,2,BS_Earnings Quality Score,BS,5


In [4]:
# desc 데이터 기준, group이 한글인 경우(재무팀에서 뽑아준 항목들) + 전체(영어 포함)를 target variable로 가져감
# target_variable = desc.loc[~desc['group'].apply(lambda x: x.encode().isalpha()),"variable"].values.tolist()

dic_group = dict()
for _, xcol in enumerate(desc['variable'].values):
    dic_group[xcol] = desc.loc[_,'group']


dic_imp = dict()
for _, xcol in enumerate(desc['variable'].values):
    dic_imp[xcol] = desc.loc[_,'importance']



dic_name = dict()
for _, xcol in enumerate(desc['raw'].values):
    dic_name[xcol] = desc.loc[_, 'variable']

dic_name['Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.1'] = "FS_Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.1"
dic_name['Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.2'] = "FS_Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.2"
dic_name['Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.3'] = "FS_Dividend Yield - Common Stock - Net - Issue Specific - %, TTM.3"

dic_name['IS_Other, Net.1'] = "IS_Other, Net.1"
dic_name['IS_Current Tax - Total.1'] = "IS_Current Tax - Total.1"
dic_name['IS_Deferred Tax - Total.1'] = "IS_Deferred Tax - Total.1"


# # C1, C2, C3, C4, C5
# C = ['매출액증가율(YoY)(연도)', 'Gross Investment (GI)','무형자산회전율s','매출총이익(천원)','무형자산'] 
# dic_c = {C[0]:"C1", C[1]: "C2", C[2]: "C3", C[3]:"C4", C[4]: "C5"}
# C_ = ['C1','C2','C3','C4','C5']

# Y1, Y2, Y3, Y4
Y = ['close','close_weighted','PBR','EV_EBITDA']

remove_col = ['인당매출','무형자산회전율','인당영업이익','인당순이익','투자비 대비 매출 (Rev/GI)']
ev_col = ['FS_Enterprise Value', 'EBITDA2(천원)', 'EBIT(천원)' ]

# Accenture

In [5]:
file_name = "Accenture_non_shift_clean_total_v1.1.csv"
company = file_name.split("_")[0]
print(company)

df = pd.read_csv(os.path.join(file_path,file_name))

final_col_lst = pd.read_excel(os.path.join(result_path,"backup",company+"_corr.xlsx"))
final_col_lst

Accenture


,cols,corr,pearson,group,importance
0,매출액(천원),0.92151,0.00000,성장성,0
1,Gross Investment (GI),0.91834,0.00000,성장성,2
2,매출총이익(천원),0.91264,0.00000,수익성,0
3,BS_Additional Paid-In Capital,0.90161,0.00000,BS,5
4,무형자산감상비 누계,-0.88885,0.00000,안정성,1
...,...,...,...,...,...
112,ROGI(%),-0.13216,0.25508,수익성,0
113,EBITDA2마진율(비율),0.12788,0.27094,수익성,1
114,매출채권회전율,0.12416,0.28525,활동성,2
115,BS_Treas Shares - Common Stock Prmry Issue,-0.11100,0.33977,BS,5


In [6]:
select_col = ['Date_x'] + df.columns[df.columns.isin(final_col_lst.cols)].values.tolist()+['EV_EBITDA']
final_df = df[select_col]
final_df

,Date_x,"BPS(지배, Adj.)(원/주)",BPS증가율(YoY)(비율),BS_Additional Paid-In Capital,BS_Contract Assets - Long Term,BS_Deferred Income Tax - Current Liability,BS_Deferred Income Tax - LT Liability,BS_Deferred Revenue - Long Term,BS_Earnings Quality Score,BS_LT Investments - Other,...,이익잉여금(천원),이자보상율,인당매출s,인당순이익s,인당영업이익s,재고자산회전율(비율),채무총계,총자산회전율(비율),현금배당성향(%),EV_EBITDA
0,2003-07-01,14.20292,-0.41881,1501,10,22,4,303,94,33,...,-642,4375.00000,0.16567,0.00618,0.01924,0.68731,60,1.98000,0.00000,4.37000
1,2003-10-01,11.97411,-0.11801,1606,10,23,4,303,94,31,...,-468,12675.00000,0.16437,0.00658,0.01963,0.68050,50,1.97000,0.00000,6.31000
2,2004-01-01,11.20423,-0.07620,1596,10,23,4,303,94,212,...,-344,2558.33333,0.16277,0.00637,0.01826,0.62587,52,1.89000,0.00000,5.33000
3,2004-04-01,7.77176,-0.36735,2151,10,23,4,303,94,309,...,-134,5209.09091,0.16209,0.00688,0.01913,0.58577,60,1.76000,0.00000,5.03000
4,2004-07-01,10.49143,-0.26132,1644,10,43,19,303,94,340,...,47,3091.66667,0.15845,0.00726,0.01862,0.62013,69,1.89000,0.00000,6.23000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2021-04-01,9.06642,0.33799,8756,40,51,278,699,97,141,...,15004,8829.16667,0.08529,0.01030,0.01284,0.62729,71,1.24000,0.35541,21.37000
72,2021-07-01,9.95228,0.22283,8618,38,51,244,700,95,145,...,13989,65300.00000,0.08538,0.01003,0.01292,0.68231,66,1.26000,0.38761,24.79000
73,2021-10-01,10.78806,0.31588,9098,42,51,320,702,90,149,...,15111,48680.00000,0.08612,0.00993,0.01309,0.80362,65,1.31000,0.33681,24.73000
74,2022-01-01,11.08288,0.25069,10066,39,51,336,747,76,156,...,16028,51550.00000,0.06674,0.00762,0.01044,0.78431,61,1.35000,0.37153,20.78000


In [7]:
final_df.to_csv(os.path.join(final_data_path, company+"_non_shift_clean_total_process_v1.1.csv"), index=False)
final_df.to_excel(os.path.join(final_data_path, company+"_non_shift_clean_total_process_v1.1.xlsx"), index=False)


# Infosys

In [8]:
file_name = "Infosys_non_shift_clean_total_v1.1.csv"
company = file_name.split("_")[0]
print(company)

df = pd.read_csv(os.path.join(file_path,file_name))

final_col_lst = pd.read_excel(os.path.join(result_path,"backup",company+"_corr.xlsx"))
final_col_lst

Infosys


,cols,corr,pearson,group,importance
0,매출액증가율(YoY)(연도),0.71562,0.00000,성장성,0.00000
1,"BPS(지배, Adj.)(원/주)",0.71313,0.00000,안정성,2.00000
2,재고자산회전율(비율),0.66231,0.00000,활동성,2.00000
3,BS_Construction in Progress - Gross,-0.66142,0.00000,BS,5.00000
4,FS_Dividend Yield - Common Stock - Net - Issue...,-0.65435,0.00000,NaN,NaN
...,...,...,...,...,...
99,CF_Unusual Items,-0.12516,0.28464,CF,5.00000
100,자본금(천원),-0.11723,0.31651,안정성,2.00000
101,FS_Free Cash Flow Net of Dividends,-0.11440,0.32842,FS,5.00000
102,CF_Cash from Financing Activities,0.10747,0.35877,CF,5.00000


In [9]:
len(df.columns[df.columns.isin(final_col_lst.cols)].values.tolist())

104

In [10]:
select_col = ['Date_x'] + df.columns[df.columns.isin(final_col_lst.cols)].values.tolist()+['EV_EBITDA']
final_df = df[select_col]
final_df

,Date_x,"BPS(지배, Adj.)(원/주)",BPS증가율(YoY)(비율),"BS_Accounts Receivable - Trade, Gross",BS_Additional Paid-In Capital,BS_Cash & Equivalents,BS_Compensation Rate - Domestic,BS_Construction in Progress - Gross,BS_Earnings Quality Score,BS_Expected Rate of Return - Domestic,...,운전자본증감(천원),유동비율(비율),유형자산감상비 누계,인당매출s,인당영업이익s,자기자본비율,자본금(천원),재고자산회전율(비율),투자비 대비 매출 (Rev/GI)s,EV_EBITDA
0,2003-07-01,10.30660,0.26789,307,206,304,0.05100,14,92,0.08200,...,-17.90000,3.89000,-151,0.06273,0.01773,82.44444,7,0.40764,1.79815,17.93000
1,2003-10-01,10.54978,0.12981,307,206,355,0.05100,18,92,0.08200,...,-2.40000,3.73000,-164,0.06150,0.01711,81.71937,7,0.40218,1.78511,21.11000
2,2004-01-01,14.69983,0.57158,307,206,398,0.05100,48,92,0.08200,...,0.70000,1.75000,-187,0.05955,0.01662,65.42373,8,0.41776,1.83884,19.62000
3,2004-04-01,15.21994,0.36756,307,206,234,0.05100,62,92,0.08200,...,-64.40000,2.99000,-188,0.05810,0.01633,80.95703,7,0.58247,1.82609,20.74000
4,2004-07-01,8.55488,-0.16996,307,206,289,0.05100,55,92,0.08200,...,-86.50000,2.56000,-196,0.05712,0.01621,77.31164,29,0.56169,1.86922,23.09000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2021-01-01,6.23239,0.29518,1754,136,2675,0.06000,145,83,0.08300,...,-775.40000,2.54000,-2063,0.05490,0.01346,70.44534,290,0.43459,1.50145,19.88000
71,2021-04-01,7.95740,0.71456,1754,149,2198,0.06000,124,94,0.08300,...,-158.00000,2.09000,-2097,0.05661,0.01399,65.64792,285,0.45523,1.54569,22.11000
72,2021-07-01,8.47161,1.14531,1754,76,1951,0.06000,69,95,0.08300,...,-505.10000,2.17000,-2120,0.05836,0.01416,65.91417,283,0.49396,1.59903,22.72000
73,2021-10-01,10.09714,0.87880,1754,90,1590,0.06000,66,97,0.08300,...,-912.70000,1.94000,-2162,0.06025,0.01432,63.62643,282,0.52258,1.65685,24.95000


In [11]:
final_df.to_csv(os.path.join(final_data_path, company+"_non_shift_clean_total_process_v1.1.csv"), index=False)
final_df.to_excel(os.path.join(final_data_path, company+"_non_shift_clean_total_process_v1.1.xlsx"), index=False)


# TCS

In [12]:
file_name = "TCS_non_shift_clean_total_v1.1.csv"
company = file_name.split("_")[0]
print(company)

df = pd.read_csv(os.path.join(file_path,file_name))

final_col_lst = pd.read_excel(os.path.join(result_path,"backup",company+"_corr.xlsx"))
final_col_lst

TCS


,cols,corr,pearson,group,importance
0,"BPS(지배, Adj.)(원/주)",0.79911,0.00000,안정성,2.00000
1,"FS_Free Cash Flow Yield - %, TTM",-0.73477,0.00000,FS,5.00000
2,FS_Dividend Yield - Common Stock - Net - Issue...,-0.63767,0.00000,FS,5.00000
3,ROGI(%),0.63102,0.00000,수익성,0.00000
4,투자비 대비 매출 (Rev/GI)s,0.61075,0.00000,성장성,0.00000
...,...,...,...,...,...
123,FS_Dividend Coverage - %,-0.11291,0.38223,FS,5.00000
124,재고자산회전율(비율),-0.11090,0.39083,활동성,2.00000
125,IS_Investment Income - Non-Operating,0.10966,0.39620,IS,5.00000
126,"IS_Other, Net",-0.10896,0.39921,IS,5.00000


In [13]:
len(df.columns[df.columns.isin(final_col_lst.cols)].values.tolist())

128

In [14]:
select_col = ['Date_x'] + df.columns[df.columns.isin(final_col_lst.cols)].values.tolist()+['EV_EBITDA']
final_df = df[select_col]
final_df

,Date_x,"BPS(지배, Adj.)(원/주)",BPS증가율(YoY)(비율),BS_Accounts Payable,"BS_Accumulated Intangible Amort, Suppl.","BS_Curr Derivative Liab. Hedging, Suppl.","BS_Goodwill, Net",BS_Income Taxes Payable,BS_Intangibles - Gross,"BS_Intangibles, Net",...,인당순이익s,인당영업이익s,자기자본비율,자본금(천원),재고자산회전율(비율),채무총계,총자산회전율(비율),투자비 대비 매출 (Rev/GI)s,현금배당성향(%),EV_EBITDA
0,2006-10-01,10.56255,-0.16945,310,8,12,285,36,65,57,...,0.01377,0.01512,68.65672,22,0.66905,13,1.88000,2.45815,0.21658,24.91000
1,2007-01-01,11.39251,0.57520,320,11,12,307,40,69,58,...,0.01374,0.01534,67.11344,23,0.64560,118,1.71000,2.33429,0.32399,23.31000
2,2007-04-01,11.21850,-0.12242,358,13,12,347,78,70,57,...,0.01407,0.01579,67.05362,24,0.62521,108,1.61000,2.22182,0.24197,20.76000
3,2007-07-01,10.37809,-0.12752,392,15,12,357,74,71,56,...,0.01419,0.01546,70.24302,25,0.59123,109,1.54000,2.11653,0.23439,18.60000
4,2008-01-01,3.32105,-0.70849,472,15,12,316,80,71,56,...,0.00987,0.01013,70.01595,24,0.61062,114,1.47000,1.42632,0.38933,13.82000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2021-04-01,13.02494,0.51162,1077,57,18,248,978,119,62,...,0.00997,0.01271,66.19664,50,0.43865,1098,1.30000,2.68529,0.64873,24.48000
58,2021-07-01,12.20749,0.65921,1103,64,21,240,929,130,65,...,0.01038,0.01323,68.15054,50,0.42512,1107,1.29000,2.76568,0.60961,26.71000
59,2021-10-01,12.63461,0.18530,982,80,10,234,966,236,156,...,0.01042,0.01319,69.17014,50,0.41217,1072,1.26000,2.49543,0.60704,25.63000
60,2022-01-01,15.06842,0.29212,1060,94,17,235,1596,240,145,...,0.01006,0.01271,62.99077,49,0.47241,1030,1.41000,2.64259,0.59998,25.04000


In [15]:
final_df.to_csv(os.path.join(final_data_path, company+"_non_shift_clean_total_process_v1.1.csv"), index=False)
final_df.to_excel(os.path.join(final_data_path, company+"_non_shift_clean_total_process_v1.1.xlsx"), index=False)
